In [1]:
import pickle
import pandas as pd
import numpy as np
import scipy
import joblib

In [2]:
models_dict = joblib.load('models_dict.joblib')
models_dict.keys()

ImportError: No module named 'sklearn.linear_model._logistic'

In [ ]:
df_cohort = pd.read_csv('sample_cohort.csv')
test_hosp, test_window, test_y = df_cohort['hosp_id'], df_cohort['window_id'], df_cohort['y']
cohort_IDs = df_cohort.set_index('ID')[[]]

In [4]:
df_cohort

,ID,hosp_id,window_id,y
0,90000-____0,90000,0,1
1,90000-____1,90000,1,1
2,90000-____2,90000,2,1
3,90001-____0,90001,0,0
4,90001-____1,90001,1,0


In [5]:
from sklearn import metrics, utils
from joblib import Parallel, delayed


def bootstrap_func(i, y_true, y_score):
    #Bootstrap resample to calculate AUROC
    yte_true_b, yte_pred_b = utils.resample(y_true, y_score, replace=True, random_state=i)
    return metrics.roc_curve(yte_true_b, yte_pred_b), metrics.roc_auc_score(yte_true_b, yte_pred_b)

def get_roc_CI(y_true, y_score):
    #Bootstrap confidence intervals 
    roc_curves, auc_scores = zip(*Parallel(n_jobs=4)(delayed(bootstrap_func)(i, y_true, y_score) for i in range(1000)))
    print('Test AUC: ({:.3f}, {:.3f}) percentile 95% CI'.format(np.percentile(auc_scores, 2.5), np.percentile(auc_scores, 97.5)))

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    for fpr, tpr, _ in roc_curves:
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        aucs.append(metrics.auc(fpr, tpr))

    mean_tpr = np.mean(tprs, axis=0)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + 1.96 * std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - 1.96 * std_tpr, 0)
    return roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper

def eval3():
    #Calculate hospital admission level AUROC for every complete window
    df_Yte = df_Yte_all.copy()
    df_Yte = df_Yte[df_Yte['window_id'] >= 1]
    df_Yte_agg = df_Yte.groupby(['hosp_id']).max()
    return df_Yte_agg

## M-CURES model performance and scores

In [3]:
#Load data and model

mcures_clfs = models_dict['M-CURES']
df_mcures = pd.read_csv('../preprocessing/sample_output/mcures.csv').set_index('ID')

NameError: name 'models_dict' is not defined

In [28]:
df_mcures

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",respiratoryrate_mask_0.0,respiratoryrate_mask_1.0,"respiratoryrate_max_(-0.001, 16.0]","respiratoryrate_max_(16.0, 18.0]","respiratoryrate_max_(18.0, 22.0]",...,_314689_value_Standing,_355444_value_Developmentally supported position,_355444_value_HOB at 15 degrees,_355444_value_HOB at 30 degrees,_355444_value_HOB at 45 degrees,_355444_value_HOB at 60 degrees,_355444_value_HOB at 90 degrees,_355444_value_HOB flat (medical condition),_355444_value_Reverse Trendelenberg,_355444_value_other (see comments)
ID,,,,,,,,,,,,,,,,,,,,,
90000-____0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90000-____1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90000-____2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90001-____0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90001-____1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#Calculate AUROC for model on given dataset


eval_matrix = scipy.sparse.csr_matrix(cohort_IDs.join(df_mcures).values.astype(float))
all_y = np.array([clf.predict_proba(eval_matrix)[:,1] for clf in mcures_clfs])

y_scores = all_y.mean(0)
df_Yte_all = pd.DataFrame({'hosp_id': test_hosp, 'window_id': test_window, 'y': test_y, 'y_score': y_scores})
df_Yte_agg = eval3()
y_score = df_Yte_agg['y_score']
y_true = df_Yte_agg['y']
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score)
print('Test AUC: {:.3f}'.format(metrics.roc_auc_score(y_true, y_score)))

# Generate 95% CI
try:
    roc_curves, auc_scores, mean_fpr, tprs_lower, tprs_upper = get_roc_CI(y_true, y_score)
except:
    pass

ValueError: X has 88 features per sample; expecting 110

In [29]:
#generate list of scores for each example 

y_score_lst = df_Yte_all.groupby(['hosp_id'])['y_score'].apply(list)
df1 = pd.DataFrame({'y_scores_mcures_lst':  y_score_lst})
df2 = pd.DataFrame({'id': df_Yte_agg.index, 'y_scores_mcures': y_score})

outcome_outputs = df1.merge(df2, left_index=True, right_index=True)

In [ ]:
outcome_outputs 